# Heavy Tailed Universality Classes



Here we demonstrate how the Heavy Tailed / Power Law Universality Classes work for correlation matrices for our layer weight matrices

$$\mathbf{X}=\mathbf{W}^{T}{W}$$

Suppose $\mathbf{W}$ is a random matrix, we choose the matrix elements $W_{i,j}=x$ from a power law (i.e. Pareto) distibution, such that 

$$p(W_{i,j})=p(x)\sim\dfrac{1}{x^{1+\mu}},\;\;\mu>0$$

### Levy and Truncated Levy Power Laws
Note: for any finite empirical density $p_{N}(x)$ (a finite sample $N$ of $p(x)$), when

- $\mu<2$, $p_{N}(x)$ remains a Power Law distribution even in the infinite limit $N\rightarrow\infty$.  This is a Levy Distribution.


- $\mu<4$, $p_{N}(x)$ is a Power Law distribution for any finite N (i.e. the tails decay very slow), but in the infinite limit $N\rightarrow\infty$, the tails $p_{N}(x)$ stop behaving like a Power law.  This is a Trucated Levy Distribution.


-  $\mu>4$, $p_{N}(x)$ has finite 4th moment, and no longer has tails that exhibit Power law decay, even at finite $N$

It turns out, the eigenvalues of $\mathbf{X}$ behave in a similar way

### Emprical Spectral Density (ESD) $p(\lambda)$

Compute the eigenvalues $\lambda$ of $\mathbf{X}$, 

$$\mathbf{X}\mathbf{v}=\lambda\mathbf{x}$$

and form a histogram representing the Emprical Spectral Density (ESD) $p(\lambda)$

It turns out, for any finite size matrix $\mathbf{W}$ (when $\mu<4$ in particular), we have

$$p(\lambda)\sim\dfrac{1}{\lambda^{\alpha}}$$

We want to understand, given $\mu$, what is $\alpha$?

The problem was discussed by Soshnikov.  Later, these results were extended, to $0<\mu<4$, by Biroli et al., with heuristic arguments and numerical simulations:  https://arxiv.org/abs/cond-mat/0609070

and also see page 26
http://www-syscom.univ-mlv.fr/~najim/gdr/bouchaud.pdf

It was proven rigorously  Auffinger et al. https://arxiv.org/pdf/0811.1587.pdf

We summarize the basic results and provide numerical simulations to help understand this


#### Power Law $\mu<2$

For small mu, we have $alpha=1+\mu/2$, such that the ESD behaves like a Power Law


 $$p(\lambda)\sim\dfrac{1}{\lambda^{1+\mu/2}}$$
   
  
#### Truncated Power Law $2<\mu<2$

When $p(x)$ is a Truncated Power Law, the ESD has a different relation, $\alpha\sim A\mu+B$

 $$p(\lambda)\sim\dfrac{1}{\lambda^{A\mu+B}}$$
 
and remains a Power Law, even for finite matrices. 

We can determine the constants $A, B$ from numerical solution (shown below)
 
#### Marchenk Pastur limiting form $\mu<4$

In this case, the ESD approaches the more familir Marchenko Pastur result from Random Matrix Theory

 $$p_{N}(\lambda)\rightarrow\rho_{mp}(\lambda)$$

although it may be difficult to observe this for any finite size, small matrix because there may be supriously large matrix elements (and therefore large eigenvalues).

#### Boundary Cases $\mu=2, 4$   

There are boundary cases for $\mu=2$ and $\mu=4$ which are interesting but I do not discuss here because it would be very difficult to observe these results for small matrices.

#### PowerLaw fitting code

https://github.com/jeffalstott/powerlaw


In [73]:
import numpy as np
import tensorflow as tf
import pickle, time
from copy import deepcopy
from shutil import copy
from tqdm import tqdm_notebook as tqdm
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb
import RMT_Util
import powerlaw

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
print(sklearn.__version__)

0.19.1


### $\mu<2$   Pareto / Power Law Distributions

Notice:
- log-log plots are farily linear past after $x_{max}\sim 1$ value
- as $\mu$ decreases, the tail fills out.

In [ ]:
mu=1.2 # Blue
w = np.random.pareto(mu,size=10000)
bins = np.logspace(np.log10(np.min(w)), np.log10(np.max(w)), num=100)
plt.hist(w, bins=bins, density=True, log=True, color='blue', alpha=0.5);
plt.xscale('log')

mu=0.8 # Red
w = np.random.pareto(mu,size=10000)
bins = np.logspace(np.log10(np.min(w)), np.log10(np.max(w)), num=100)
plt.hist(w, bins=bins, density=True, log=True, color='red', alpha=0.25);
plt.xscale('log')

mu=0.4  # Yellow
w = np.random.pareto(mu,size=10000)
bins = np.logspace(np.log10(np.min(w)), np.log10(np.max(w)), num=100)
plt.hist(w, bins=bins, density=True, log=True, color='yellow', alpha=0.25);
plt.xscale('log')
plt.show()

### $\mu=0.4$ ESD

The tail remains linear

In [ ]:
mu=0.4
W = np.random.pareto(mu,size=(300,1000))
Q = np.max(W.shape)/np.min(W.shape)
evals, _ = RMT_Util.eigenspectrum(W)
bins = np.logspace(np.log10(np.min(evals)), np.log10(np.max(evals)), num=100)
plt.hist(evals, bins=bins, density=True, log=True, color='blue', alpha=0.25);
plt.xscale('log')
plt.title(r'$\rho(\lambda)$  $\mu={}$'.format(mu))
plt.show()

#### PowerLaw fit  $\mu=0.4$  too small

The PowerLaw method does not work well here..it over-estimates $\alpha$, as expected, and it mis-identifies the distribution as a Truncated power law.


In [ ]:
alpha, D, best_pl = RMT_Util.fit_powerlaw(evals)
print("alpha {:3g}, D {:3g}, best_pl  {}".format(alpha, D, best_pl))

### $\mu=1.2$ ESD

The tail remains fat and the log-log plot is linear


In [ ]:
mu=1.2
W = np.random.pareto(mu,size=(300,1000))
Q = np.max(W.shape)/np.min(W.shape)
evals, _ = RMT_Util.eigenspectrum(W)
bins = np.logspace(np.log10(np.min(evals)), np.log10(np.max(evals)), num=100)
plt.hist(evals, bins=bins, density=True, log=True, color='blue', alpha=0.25);
plt.xscale('log')
plt.title(r'$\rho(\lambda)$  $\mu={}$'.format(mu))
plt.show()

#### Power Law fit:  much better

For small $\mu$, the PowerLaw method consistantly returns a best fit to be Truncated Power Law (TPL)


In [ ]:
alpha, D, best_pl = RMT_Util.fit_powerlaw(evals)
print("alpha {:3g}, D {:3g}, best_pl  {}".format(alpha, D, best_pl))

In [ ]:
fit = powerlaw.Fit(evals, xmax=np.max(evals))
R, p = fit.distribution_compare('truncated_power_law', 'power_law', normalized_ratio=True)
R, p

In [ ]:
fig2 = fit.plot_pdf(color='b', linewidth=2)
fit.power_law.plot_pdf(color='b', linestyle='--', ax=fig2)
fit.plot_ccdf(color='r', linewidth=2, ax=fig2)
fit.power_law.plot_ccdf(color='r', linestyle='--', ax=fig2)
plt.title(r'fit of $\mu=1.2$ ESD')
plt.show()


### $2<mu<4$

- The log-log plot is nearly linear

- but the tail drops off faster than linear

- The PowerLaw method consistantly returns a best fit to be Power Law (PL)

- The PowerLaw graphs cuts off for large eigenvalues


In [ ]:
mu=2.5
W = np.random.pareto(mu,size=(300,1000))
Q = np.max(W.shape)/np.min(W.shape)
evals, _ = RMT_Util.eigenspectrum(W)
bins = np.logspace(np.log10(np.min(evals)), np.log10(np.max(evals)), num=100)
plt.hist(evals, bins=bins, density=True, log=True, color='blue', alpha=0.25);
plt.xscale('log')
plt.title(r'$\rho(\lambda)$  $\mu={}$'.format(mu))
plt.show()

RMT_Util.fit_and_plot_powerlaw(evals)

In [ ]:
mu=3.5
W = np.random.pareto(mu,size=(300,1000))
Q = np.max(W.shape)/np.min(W.shape)
evals, _ = RMT_Util.eigenspectrum(W)
bins = np.logspace(np.log10(np.min(evals)), np.log10(np.max(evals)), num=100)
plt.hist(evals, bins=bins, density=True, log=True, color='blue', alpha=0.25);
plt.xscale('log')
plt.title(r'$\rho(\lambda)$  $\mu={}$'.format(mu))
plt.show() 

RMT_Util.fit_and_plot_powerlaw(evals)


### $4<mu$


In [ ]:
mu=5.0
W = np.random.pareto(mu,size=(300,1000))
Q = np.max(W.shape)/np.min(W.shape)
evals, _ = RMT_Util.eigenspectrum(W)
bins = np.logspace(np.log10(np.min(evals)), np.log10(np.max(evals)), num=100)
plt.hist(evals, bins=bins, density=True, log=True, color='blue', alpha=0.25);
plt.xscale('log')
plt.title(r'$\rho(\lambda)$  $\mu={}$'.format(mu))
plt.show() 

RMT_Util.fit_and_plot_powerlaw(evals)

### Overlay $\mu=1.0$ and $\mu=3.0$ to compare how the ESDs decay

In [ ]:
W = np.random.pareto(1.0,size=(3000,9000))
W = W/100
Q = np.max(W.shape)/np.min(W.shape)
evals_small, _ = RMT_Util.eigenspectrum(W)

W = np.random.pareto(3.0,size=(3000,9000))
W = W
Q = np.max(W.shape)/np.min(W.shape)
evals_medium, _ = RMT_Util.eigenspectrum(W)

bins = np.logspace(np.log10(np.min(evals_small)), np.log10(np.max(evals_small)), num=100)
plt.hist(evals_small, bins=bins, density=True, log=True, color='blue', alpha=0.25, label="$\mu=1$");
plt.xscale('log')
#plt.show()

bins = np.logspace(np.log10(np.min(evals_medium)), np.log10(np.max(evals_medium)), num=100)
plt.hist(evals_medium, bins=bins, density=True, log=True, color='green', alpha=0.25, label="$\mu=3$");
plt.xscale('log')
plt.title("Comparison of Log-log Histogram for different heavy tailed matrix ESDs")
plt.legend()

# plt.savefig("img/heavy-tailed-log-log-esds.png")

In [ ]:
bins = np.logspace(np.log10(np.min(evals_medium)), np.log10(np.max(evals_medium)), num=100)
plt.hist(evals_medium, bins=bins, density=True, log=True, color='green', alpha=0.25, label="$\mu=3$");
plt.xscale('log')
plt.title("Log-log Histogram for $\mu=3$, Zoomed in")
plt.legend()

# plt.savefig("img/heavy-tailed-log-log-esd-zoomed.png")

### ESDs  Fits, comparison to Theory

This is pretty slow (> 24 hours), but it will let us see how $\mu$ relates to $\alpha$ for each Universality Class

In [2]:
def get_pareto_mat(mu, M=5000, Q=2):
    N = int(Q*M)
    W = np.random.pareto(mu,size=(M,N))
    evals, _ = RMT_Util.eigenspectrum(W)
    return W, evals

In [ ]:
alphas = []
#mus = [1.5, 1.8, 2.0, 2.2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6]
mus = np.sort(3*np.random.random_sample(100)+.5)
for mu in tqdm(mus):
      
    W, evals = get_pareto_mat(mu) 
    
    alpha, _, _ = RMT_Util.fit_powerlaw(evals)
    alphas.append(alpha)


In [ ]:
small_mu_alphas = []
medium_mu_alphas = []
for mu in mus:
    alpha = 1.0+mu/2.0
    small_mu_alphas.append(alpha)
    
    alpha = 2.5*(mu)-3
    medium_mu_alphas.append(alpha)


In [ ]:
plt.scatter(mus, alphas, marker='.')  
plt.plot(mus, small_mu_alphas, color='red')
plt.plot(mus, medium_mu_alphas, color='orange')
plt.xlabel('$\mu$')
plt.ylabel(r'$\alpha$')
plt.title(r'Powerlaw estimate for $\alpha$ given $\mu$ ')